In [88]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import pyLDAvis.sklearn
import re
import sys
import wikipedia

print("Ok")

Ok


In [89]:
def get_topic(topic):
    pages = wikipedia.search(topic, results = 50, suggestion = True)[0]
    documents = []
    document_names = []
    for i in pages:
        print(i)
        try:
            page = wikipedia.page(i)
            documents.append(page.content)
            document_names.append(i)
        except Exception as e:
            print(e, file=sys.stderr)
    return documents, document_names

In [90]:

def clean_text(text):
    tokenized_text = word_tokenize(text.lower())
    cleaned_text = [t for t in tokenized_text if t not in stopwords.words('english') and re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', t)]
    return cleaned_text


def stem_text(text):
    ps = PorterStemmer()
    ret = []
    for word in text:
        ret.append(ps.stem(word))
    return ret


def join_text(text):
    return " ".join(text)

def get_documents_for_topic(topic):
    docs, doc_names = get_topic(topic)
    docs = tuple(map(clean_text, docs))
    docs = tuple(map(stem_text, docs))
    docs = tuple(map(join_text, docs))
    return docs, doc_names

<>:3: DeprecationWarning: invalid escape sequence '\-'
<>:3: DeprecationWarning: invalid escape sequence '\-'
C:\Users\GeorgeS\AppData\Local\Temp\ipykernel_17324\3257789200.py:3: DeprecationWarning: invalid escape sequence '\-'
  cleaned_text = [t for t in tokenized_text if t not in stopwords.words('english') and re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', t)]


In [91]:

docs, doc_names = get_documents_for_topic("Machine Learning in Natural Language Processing")

Natural language processing
History of natural language processing
Natural Language Toolkit
Natural-language understanding
Transformer (machine learning model)
Machine learning


Page id "machine ;earning" does not match any pages. Try another id!


Deep learning
ID3 algorithm
List of artificial intelligence projects
Zero-shot learning
Augmented Analytics


Page id "augmented analysis" does not match any pages. Try another id!


Natural language generation
BERT (language model)
Outline of natural language processing
Google Brain
Active learning (machine learning)
Wendy Lehnert
Stop word
ComplyAdvantage


...

Federated learning
GPT-3


Page id "gtp 3" does not match any pages. Try another id!


Apache OpenNLP
Semantic decomposition (natural language processing)
Cognitive computing
Quantum machine learning
Self-supervised learning
Deep linguistic processing
Adversarial machine learning
List of datasets for machine-learning research
Hugging Face


Page id "huging face" does not match any pages. Try another id!


Language acquisition
Reinforcement learning
Empirical Methods in Natural Language Processing
Mohamed bin Zayed University of Artificial Intelligence
Machine translation
Recorded Future
Document processing
Outline of machine learning
Timeline of machine learning
Pachinko allocation
Language model
Language identification
GloVe
Grammar induction
Semantic analysis (machine learning)
Never-Ending Language Learning
Machine learning in bioinformatics
Gensim
List of programming languages for artificial intelligence
Word embedding


## Bag of words

In [92]:
def print_vectorizer_method(vectorizer, X, method):
    print(f"Method {method}")
    print("Shape: documents/words", X.shape)
    features_bag_of_words = vectorizer.get_feature_names_out()
    print(f"Features length is {len(features_bag_of_words)}:", features_bag_of_words)
    print('\n')

def create_bag_of_words_vectorizer(docs):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(docs)
    print_vectorizer_method(vectorizer, X, "Bag of Words")
    return vectorizer, X


def create_tf_idf_vectorizer(docs):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(docs)
    print_vectorizer_method(vectorizer, X, "TF IDF")
    return vectorizer, X


def create_vectorizer(docs, name):
    if name == "bag_of_words":
        return create_bag_of_words_vectorizer(docs)
    if name == "tf_idf":
        return create_tf_idf_vectorizer(docs)
    raise f"Not implemented, {name}"


create_vectorizer(docs, "bag_of_words")
create_vectorizer(docs, "tf_idf")
print("Done")

Method Bag of Words
Shape: documents/words (45, 6258)
Features length is 6258: ['0011526042365591' '0033' '009' ... 'zsl' 'zurada' 'zurich']


Method TF IDF
Shape: documents/words (45, 6258)
Features length is 6258: ['0011526042365591' '0033' '009' ... 'zsl' 'zurada' 'zurich']


Done


In [93]:
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print(f"Topic {idx}")
        print([(vectorizer.get_feature_names_out()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
    print('\n')

In [94]:
def create_model(n_components, X, model):
    if model == "SVD":
        model = TruncatedSVD(n_components=n_components)
    elif model == "NMF":
        model = NMF(n_components=n_components)
    else:
        raise f"{model} does not exist"

    return model, model.fit_transform(X)


In [95]:
NUM_TOPICS = 5

def create_and_print(docs, topics,method, factorizer):
    vectorizer, X = create_vectorizer(docs, method)
    model, fitted = create_model(topics, X, factorizer)
    print(fitted.shape)
    print(fitted[0])
    print_topics(model, vectorizer)
    return vectorizer, X, fitted


In [96]:
create_and_print(docs, NUM_TOPICS, "bag_of_words", "SVD")

Method Bag of Words
Shape: documents/words (45, 6258)
Features length is 6258: ['0011526042365591' '0033' '009' ... 'zsl' 'zurada' 'zurich']


(45, 5)
[131.58904347  50.21249044  11.57746624  -7.30621593   9.15464329]
Topic 0
[('learn', 0.4222048965871562), ('languag', 0.3932539096636138), ('use', 0.23133639891520447), ('machin', 0.19919767453965118), ('model', 0.19886750878350606), ('comput', 0.14687382141186475), ('data', 0.14414836971271344), ('process', 0.14378820542602858), ('translat', 0.14257049501101787), ('word', 0.12682523557777944)]
Topic 1
[('languag', 0.5925319415741599), ('translat', 0.19524263967564387), ('word', 0.16165770193574094), ('text', 0.12317378633057023), ('acquisit', 0.10212789434293139), ('linguist', 0.09977986007013427), ('children', 0.083183911958335), ('natural', 0.08203474077947023), ('human', 0.07964457767705851), ('grammar', 0.06830680594439101)]
Topic 2
[('translat', 0.6528689235919222), ('machin', 0.2926536538524599), ('text', 0.15122557074683055), ('

(CountVectorizer(),
 <45x6258 sparse matrix of type '<class 'numpy.int64'>'
 	with 21019 stored elements in Compressed Sparse Row format>,
 array([[ 1.31589043e+02,  5.02124904e+01,  1.15774662e+01,
         -7.30621593e+00,  9.15464329e+00],
        [ 2.83151646e+01,  5.84454584e+00,  8.79324618e+00,
          1.29349843e+00, -2.30918629e+00],
        [ 6.11141969e+00,  2.89424282e+00,  1.38962726e-01,
         -1.65392130e-02,  2.82769868e-01],
        [ 3.68931872e+01,  2.23143268e+01,  8.66568191e-01,
         -1.37133159e+00,  8.43436271e+00],
        [ 5.18839528e+01, -8.37679423e+00,  6.52931374e+00,
         -1.49806814e+01,  9.66174594e+00],
        [ 2.32236723e+02, -1.30566757e+02,  8.33935959e+00,
         -1.23471073e+02,  1.00967250e+02],
        [ 2.11410003e+01, -1.08371789e+01,  1.33208436e+00,
          2.65603180e+00, -1.36388495e+01],
        [ 3.42238519e+01, -1.79309685e+00,  2.56997391e+00,
         -4.10924607e+00,  4.82363703e+00],
        [ 2.58099639e+01, -5.

In [97]:

output_notebook()

C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\bokeh\io\notebook.py:487: DeprecationWarning: The `source` parameter emit a  deprecation warning since IPython 8.0, it had no effects for a long time and will  be removed in future versions.
  publish_display_data(data, metadata, source, transient=transient, **kwargs)


Loading BokehJS ...

C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\bokeh\io\notebook.py:487: DeprecationWarning: The `source` parameter emit a  deprecation warning since IPython 8.0, it had no effects for a long time and will  be removed in future versions.
  publish_display_data(data, metadata, source, transient=transient, **kwargs)


In [98]:

def plot(method, data):
    df = pd.DataFrame(columns=['x', 'y', 'index'])
    df['x'], df['y'], df['index'] = method[:,0], method[:,1], data

    source = ColumnDataSource(ColumnDataSource.from_df(df))

    labels = LabelSet(x="x", y="y", text="index", y_offset=8,
                      text_font_size="8pt", text_color="#555555",
                      source=source, text_align='center')

    plot = figure(plot_width=600, plot_height=600)
    plot.circle("x", "y", size=12, source=source, line_color="black",
                fill_alpha=0.8)
    plot.add_layout(labels)
    show(plot, notebook_handle=True)

def plot2(data, vectorizer):
    svd = TruncatedSVD(n_components=2)
    words_2d = svd.fit_transform(data.T)

    df = pd.DataFrame(columns=['x', 'y', 'word'])
    df['x'], df['y'], df['word'] = words_2d[:,0], words_2d[:,1], vectorizer.get_feature_names()

    source = ColumnDataSource(ColumnDataSource.from_df(df))
    labels = LabelSet(x="x", y="y", text="word", y_offset=8,
                      text_font_size="8pt", text_color="#555555",
                      source=source, text_align='center')

    plot = figure(plot_width=600, plot_height=600)
    plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
    plot.add_layout(labels)
    show(plot, notebook_handle=True)


def create_and_print_and_plot(docs, doc_names, topics,method, factorizer):
    vectorizer, X, fitted = create_and_print(docs, topics, method, factorizer)
    plot(fitted, doc_names)
    plot2(X, vectorizer)

In [99]:
create_and_print_and_plot(docs, doc_names, NUM_TOPICS, "bag_of_words", "SVD")

Method Bag of Words
Shape: documents/words (45, 6258)
Features length is 6258: ['0011526042365591' '0033' '009' ... 'zsl' 'zurada' 'zurich']


(45, 5)
[131.58904357  50.21248875  11.57700339  -7.30131065   9.15523196]
Topic 0
[('learn', 0.4222048967034296), ('languag', 0.3932539093999371), ('use', 0.2313363987340108), ('machin', 0.19919767450077502), ('model', 0.19886750832953248), ('comput', 0.14687382136937505), ('data', 0.14414836976743642), ('process', 0.14378820564628397), ('translat', 0.1425704950241156), ('word', 0.12682523562315812)]
Topic 1
[('languag', 0.5925319272010949), ('translat', 0.19524263739955183), ('word', 0.16165775087175907), ('text', 0.1231737862891163), ('acquisit', 0.10212789825135099), ('linguist', 0.09977986495276488), ('children', 0.08318391514578767), ('natural', 0.08203474952871173), ('human', 0.07964457656090479), ('grammar', 0.06830679683212128)]
Topic 2
[('translat', 0.6528688112127259), ('machin', 0.2926537387122905), ('text', 0.15122641290902644), ('u

C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\bokeh\io\notebook.py:487: DeprecationWarning: The `source` parameter emit a  deprecation warning since IPython 8.0, it had no effects for a long time and will  be removed in future versions.
  publish_display_data(data, metadata, source, transient=transient, **kwargs)


C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\bokeh\io\notebook.py:487: DeprecationWarning: The `source` parameter emit a  deprecation warning since IPython 8.0, it had no effects for a long time and will  be removed in future versions.
  publish_display_data(data, metadata, source, transient=transient, **kwargs)


C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\bokeh\io\notebook.py:487: DeprecationWarning: The `source` parameter emit a  deprecation warning since IPython 8.0, it had no effects for a long time and will  be removed in future versions.
  publish_display_data(data, metadata, source, transient=transient, **kwargs)


C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\bokeh\io\notebook.py:487: DeprecationWarning: The `source` parameter emit a  deprecation warning since IPython 8.0, it had no effects for a long time and will  be removed in future versions.
  publish_display_data(data, metadata, source, transient=transient, **kwargs)


In [100]:
create_and_print_and_plot(docs, doc_names, NUM_TOPICS, "tf_idf", "SVD")

Method TF IDF
Shape: documents/words (45, 6258)
Features length is 6258: ['0011526042365591' '0033' '009' ... 'zsl' 'zurada' 'zurich']


(45, 5)
[ 0.73431886 -0.27870096  0.06037902 -0.03885453 -0.10052946]
Topic 0
[('languag', 0.29533578941218463), ('learn', 0.2479156910669304), ('use', 0.1947819511058322), ('word', 0.15591079241546135), ('model', 0.1558312942490691), ('process', 0.13782834924608314), ('system', 0.13427071735294413), ('machin', 0.13422719327485988), ('data', 0.13243941684288993), ('translat', 0.12032628884257603)]
Topic 1
[('learn', 0.33865310662762965), ('data', 0.17407754285046964), ('algorithm', 0.15299129330721956), ('label', 0.13869022258461303), ('dataset', 0.1226947833371157), ('attack', 0.11469861234496774), ('feder', 0.1111661668741523), ('quantum', 0.1080553679835218), ('model', 0.10635025813323154), ('node', 0.10456195272339439)]
Topic 2
[('word', 0.35462772867851594), ('bert', 0.19917532568303947), ('embed', 0.19043844353892636), ('model', 0.17349223625793

C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\bokeh\io\notebook.py:487: DeprecationWarning: The `source` parameter emit a  deprecation warning since IPython 8.0, it had no effects for a long time and will  be removed in future versions.
  publish_display_data(data, metadata, source, transient=transient, **kwargs)


C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\bokeh\io\notebook.py:487: DeprecationWarning: The `source` parameter emit a  deprecation warning since IPython 8.0, it had no effects for a long time and will  be removed in future versions.
  publish_display_data(data, metadata, source, transient=transient, **kwargs)


C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\bokeh\io\notebook.py:487: DeprecationWarning: The `source` parameter emit a  deprecation warning since IPython 8.0, it had no effects for a long time and will  be removed in future versions.
  publish_display_data(data, metadata, source, transient=transient, **kwargs)


C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\bokeh\io\notebook.py:487: DeprecationWarning: The `source` parameter emit a  deprecation warning since IPython 8.0, it had no effects for a long time and will  be removed in future versions.
  publish_display_data(data, metadata, source, transient=transient, **kwargs)


In [101]:
create_and_print_and_plot(docs, doc_names, NUM_TOPICS, "bag_of_words", "NMF")

Method Bag of Words
Shape: documents/words (45, 6258)
Features length is 6258: ['0011526042365591' '0033' '009' ... 'zsl' 'zurada' 'zurich']


(45, 5)
[0.33026179 2.50249587 2.29463152 0.06211282 0.92997122]
Topic 0
[('learn', 15.402628169872957), ('model', 8.076199277666367), ('data', 6.525116780339224), ('algorithm', 6.146797779494227), ('machin', 5.262857804371684), ('feder', 3.9528858156892785), ('use', 3.935595490456424), ('local', 3.7228213375796733), ('node', 3.3143672664698447), ('cluster', 2.9417397052856944)]
Topic 1
[('languag', 25.289826432394566), ('learn', 6.898022251252406), ('word', 6.684759391265246), ('acquisit', 5.555643961858254), ('process', 4.7121448636360315), ('children', 4.498684576669163), ('use', 4.38058973731216), ('speech', 3.938276215714694), ('human', 3.897272244715339), ('linguist', 3.612192091143066)]
Topic 2
[('translat', 14.898577450354693), ('machin', 7.408474072757111), ('languag', 5.811092361376236), ('use', 4.611602543029759), ('text', 4.420669831

C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\bokeh\io\notebook.py:487: DeprecationWarning: The `source` parameter emit a  deprecation warning since IPython 8.0, it had no effects for a long time and will  be removed in future versions.
  publish_display_data(data, metadata, source, transient=transient, **kwargs)


C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\bokeh\io\notebook.py:487: DeprecationWarning: The `source` parameter emit a  deprecation warning since IPython 8.0, it had no effects for a long time and will  be removed in future versions.
  publish_display_data(data, metadata, source, transient=transient, **kwargs)


C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\bokeh\io\notebook.py:487: DeprecationWarning: The `source` parameter emit a  deprecation warning since IPython 8.0, it had no effects for a long time and will  be removed in future versions.
  publish_display_data(data, metadata, source, transient=transient, **kwargs)


C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\bokeh\io\notebook.py:487: DeprecationWarning: The `source` parameter emit a  deprecation warning since IPython 8.0, it had no effects for a long time and will  be removed in future versions.
  publish_display_data(data, metadata, source, transient=transient, **kwargs)


In [102]:
create_and_print_and_plot(docs, doc_names, NUM_TOPICS, "tf_idf", "NMF")


Method TF IDF
Shape: documents/words (45, 6258)
Features length is 6258: ['0011526042365591' '0033' '009' ... 'zsl' 'zurada' 'zurich']


(45, 5)
[0.43877869 0.05825742 0.10026808 0.01358966 0.0898332 ]
Topic 0
[('languag', 0.7186116074701042), ('translat', 0.3049536596207494), ('text', 0.28485633054893933), ('system', 0.25125666242115485), ('natural', 0.2511645658454375), ('grammar', 0.250908700005121), ('linguist', 0.2426940743537118), ('use', 0.2214565530316611), ('process', 0.21978335717013345), ('word', 0.17691633658967118)]
Topic 1
[('learn', 0.4935229593112749), ('data', 0.26210826713228247), ('algorithm', 0.23608072179498177), ('model', 0.18601746047253576), ('use', 0.17109856204670554), ('label', 0.1691799809641371), ('dataset', 0.16158446166249488), ('exampl', 0.14352350612182121), ('machin', 0.13932765633246671), ('node', 0.1327518728750316)]
Topic 2
[('word', 0.5129167342710286), ('bert', 0.3469464079219475), ('embed', 0.27001518059799867), ('model', 0.24099882479907514), ('

C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\bokeh\io\notebook.py:487: DeprecationWarning: The `source` parameter emit a  deprecation warning since IPython 8.0, it had no effects for a long time and will  be removed in future versions.
  publish_display_data(data, metadata, source, transient=transient, **kwargs)


C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\bokeh\io\notebook.py:487: DeprecationWarning: The `source` parameter emit a  deprecation warning since IPython 8.0, it had no effects for a long time and will  be removed in future versions.
  publish_display_data(data, metadata, source, transient=transient, **kwargs)


C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\bokeh\io\notebook.py:487: DeprecationWarning: The `source` parameter emit a  deprecation warning since IPython 8.0, it had no effects for a long time and will  be removed in future versions.
  publish_display_data(data, metadata, source, transient=transient, **kwargs)


C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\bokeh\io\notebook.py:487: DeprecationWarning: The `source` parameter emit a  deprecation warning since IPython 8.0, it had no effects for a long time and will  be removed in future versions.
  publish_display_data(data, metadata, source, transient=transient, **kwargs)


In [103]:
def LDA(docs, n_components, method):
    vectorizer, X = create_vectorizer(docs, method)
    model = LatentDirichletAllocation(n_components=n_components, max_iter=10, learning_method='online')
    lda = model.fit_transform(X)
    return model, vectorizer, X


def create_panel(docs, n_components, method):
    model, vectorizer, X = LDA(docs, n_components, method)
    pyLDAvis.enable_notebook()
    panel = pyLDAvis.sklearn.prepare(model, X, vectorizer, mds='mmds')
    return panel


In [104]:

create_panel(docs, NUM_TOPICS, "bag_of_words")


Method Bag of Words
Shape: documents/words (45, 6258)
Features length is 6258: ['0011526042365591' '0033' '009' ... 'zsl' 'zurada' 'zurich']




C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.112847 -0.037254       1        1  57.823533
2      0.058311  0.070055       2        1  33.014485
0     -0.021726 -0.066741       3        1   7.745321
4     -0.065708  0.029262       4        1   1.400312
1     -0.083724  0.004677       5        1   0.016349, topic_info=          Term        Freq       Total Category  logprob  loglift
3134     learn  905.000000  905.000000  Default  30.0000  30.0000
3084   languag  803.000000  803.000000  Default  29.0000  29.0000
3601     model  488.000000  488.000000  Default  28.0000  28.0000
1380      data  356.000000  356.000000  Default  27.0000  27.0000
5959       use  558.000000  558.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
3775   network    0.005966  247.787662   Topic5  -7.3207  -1.9155
1515   develop    0.005077  147.848580   Topic5  -7.4820  -1.5604
4735  research    0.005339  186.044250   Topic5  -7.4317  -1.7399
5620      text    0.005352  230.392085   Topic5  -7.4293  -1.9513
2599     human    0.005160  206.323477   Topic5  -7.4658  -1.8775

[381 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
109       1  0.950756    acquir
109       2  0.022637    acquir
109       3  0.022637    acquir
110       1  0.966663  acquisit
110       2  0.025108  acquisit
...     ...       ...       ...
6176      2  0.033325      word
6176      3  0.027266      word
6176      4  0.003030      word
6195      1  0.933238   written
6195      2  0.051847   written

[592 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 1, 5, 2])

In [105]:
create_panel(docs, NUM_TOPICS, "tf_idf")

Method TF IDF
Shape: documents/words (45, 6258)
Features length is 6258: ['0011526042365591' '0033' '009' ... 'zsl' 'zurada' 'zurich']




C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.002515  0.006070       1        1  91.402077
1     -0.000447 -0.001772       2        1   2.927659
0     -0.001058 -0.001615       3        1   2.537413
4     -0.000973 -0.001028       4        1   1.566807
3     -0.000037 -0.001654       5        1   1.566043, topic_info=          Term      Freq     Total Category  logprob  loglift
4994    semant  0.000000  0.000000  Default  30.0000  30.0000
3300    machin  0.000000  0.000000  Default  29.0000  29.0000
4440   process  0.000000  0.000000  Default  28.0000  28.0000
2815  intellig  0.000000  0.000000  Default  27.0000  27.0000
5917   univers  0.000000  0.000000  Default  26.0000  26.0000
...        ...       ...       ...      ...      ...      ...
2290    gensim  0.001596  0.171626   Topic5  -8.5568  -0.5212
707   boundari  0.001537  0.096490   Topic5  -8.5947   0.0167
1536   dialogu  0.001530  0.088733   Topic5  -8.5993   0.0960
3300    machin  0.001540  0.375280   Topic5  -8.5926  -1.3394
3177     level  0.001534  0.140499   Topic5  -8.5965  -0.3609

[318 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
3084      1  1.440555  languag
3134      1  1.523088    learn
5959      1  1.929518      use, R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1, 5, 4])

We observe that 1:
* a
* b
* c


In [106]:
def all_in_one(topic):
    num_topics = 5
    docs, doc_names = get_documents_for_topic(topic)
    create_and_print_and_plot(docs, doc_names, num_topics, "bag_of_words", "SVD")
    create_and_print_and_plot(docs, doc_names, num_topics, "tf_idf", "SVD")
    create_and_print_and_plot(docs, doc_names, num_topics, "bag_of_words", "NMF")
    create_and_print_and_plot(docs, doc_names, num_topics, "tf_idf", "NMF")
    p1 = create_panel(docs, num_topics, "bag_of_words")
    p2 = create_panel(docs, num_topics, "tf_idf")
    return p1, p2  # panels must be evaluated to be shown in jupyter

In [107]:
p1, p2 = all_in_one("Machine Learning in Medicine")


Machine learning


Page id "machine ;earning" does not match any pages. Try another id!


Logic learning machine
Deep learning
List of datasets for machine-learning research
Federated learning
Personalized medicine
Machine learning in bioinformatics
Jordan Harrod
Ensemble learning
Fei-Fei Li
Causal inference
Suchi Saria
Artificial intelligence in healthcare
Learning


Page id "learnning" does not match any pages. Try another id!


Explainable artificial intelligence
Medicine
Rote learning
Applications of artificial intelligence
Data mining


...

C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\bokeh\io\notebook.py:487: DeprecationWarning: The `source` parameter emit a  deprecation warning since IPython 8.0, it had no effects for a long time and will  be removed in future versions.
  publish_display_data(data, metadata, source, transient=transient, **kwargs)


Method Bag of Words
Shape: documents/words (45, 7996)
Features length is 7996: ['00' '00010' '0003' ... 'zone' 'zones' 'zuckerberg']




C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


Method TF IDF
Shape: documents/words (45, 7996)
Features length is 7996: ['00' '00010' '0003' ... 'zone' 'zones' 'zuckerberg']




C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [108]:
p1

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.080858  0.123146       1        1  62.544371
0     -0.000333 -0.069734       2        1  12.942548
4     -0.083202 -0.018533       3        1  10.204526
1      0.091111 -0.093317       4        1   7.712382
2      0.073282  0.058438       5        1   6.596173, topic_info=         Term        Freq       Total Category  logprob  loglift
4350  medicin  453.000000  453.000000  Default  30.0000  30.0000
2224     educ  359.000000  359.000000  Default  29.0000  29.0000
4346    medic  382.000000  382.000000  Default  28.0000  28.0000
7954     year  284.000000  284.000000  Default  27.0000  27.0000
274    altern  164.000000  164.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
3480   includ   15.963708  244.940288   Topic5  -5.8755  -0.0120
1040   cancer   11.044665   81.547054   Topic5  -6.2438   0.7194
271      also   12.022723  341.463858   Topic5  -6.1590  -0.6278
4964    often   10.628720  116.740989   Topic5  -6.2822   0.3223
6983   system   10.859199  365.200328   Topic5  -6.2608  -0.7968

[357 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
91        1  0.948503  accuraci
91        2  0.016640  accuraci
91        3  0.016640  accuraci
91        5  0.016640  accuraci
95        1  0.089208      acgm
...     ...       ...       ...
7954      1  0.116115      year
7954      2  0.351864      year
7954      3  0.063336      year
7954      4  0.457423      year
7954      5  0.014075      year

[1023 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 5, 2, 3])

In [109]:
p2

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.004041 -0.003161       1        1  93.358995
2     -0.001023  0.001178       2        1   2.274783
0     -0.001113  0.000174       3        1   1.456174
3     -0.001364  0.000717       4        1   1.455044
4     -0.000542  0.001091       5        1   1.455004, topic_info=         Term      Freq     Total Category  logprob  loglift
4350  medicin  0.000000  0.000000  Default  30.0000  30.0000
7489  univers  0.000000  0.000000  Default  29.0000  29.0000
7579      use  0.000000  0.000000  Default  28.0000  28.0000
4170   machin  0.000000  0.000000  Default  27.0000  27.0000
1744     data  0.000000  0.000000  Default  26.0000  26.0000
...       ...       ...       ...      ...      ...      ...
5143    paper  0.001225  0.089796   Topic5  -8.8313  -0.0644
749    better  0.001227  0.108453   Topic5  -8.8299  -0.2518
7313    trial  0.001230  0.157759   Topic5  -8.8269  -0.6236
3968    learn  0.001230  0.564024   Topic5  -8.8272  -1.8979
3720     item  0.001227  0.121254   Topic5  -8.8299  -0.3633

[325 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
3968      1  1.772975  learn
7579      1  1.929925    use, R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1, 4, 5])

We observe that 2:
* a
* b
* c

In [110]:
p1, p2 = all_in_one("Shadows of the past")


A Shadow of the Past
The Shadow of the Past
Shadows of the Past
Shadows of the Past (album)
La sombra del pasado
Shadow of the Past
Sentenced
The Shadows
Shadows of the Past (1991 film)
Shadows of the Past (1922 film)


Page id "shadows of the past 1936 film" does not match any pages. Try another id!


Shadows of Forgotten Ancestors
Shadows of the Past (1936 film)
Frank Chase (screenwriter)
Erika Anderson
Stronger Than Ever (album)
Rings of Power
Christopher Paolini


Page id "christopher paola i" does not match any pages. Try another id!


Shadow Chancellor of the Exchequer
Dark Shadows
Politics of India
The Shadow of Her Past
Fátima Guedes
C tuning (guitar)
Lords of the Trident
List of The Hobbit characters
Abilene (film)


...

C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\bokeh\io\notebook.py:487: DeprecationWarning: The `source` parameter emit a  deprecation warning since IPython 8.0, it had no effects for a long time and will  be removed in future versions.
  publish_display_data(data, metadata, source, transient=transient, **kwargs)


Method Bag of Words
Shape: documents/words (44, 9124)
Features length is 9124: ['10' '15' '150m' ... 'zucco' 'zurita' 'zwangobani']




C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


Method TF IDF
Shape: documents/words (44, 9124)
Features length is 9124: ['10' '15' '150m' ... 'zucco' 'zurita' 'zwangobani']




C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\GeorgeS\.conda\envs\new_env\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [111]:
p1

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.115387 -0.038405       1        1  55.428354
2      0.054150  0.072676       2        1  17.621462
1     -0.052683 -0.096913       3        1  12.528284
4     -0.095025  0.054988       4        1  12.222067
0     -0.021829  0.007654       5        1   2.199833, topic_info=         Term        Freq       Total Category  logprob  loglift
2963     film  447.000000  447.000000  Default  30.0000  30.0000
6969  saffron  109.000000  109.000000  Default  29.0000  29.0000
420    aragon  101.000000  101.000000  Default  28.0000  28.0000
6810     ring  635.000000  635.000000  Default  27.0000  27.0000
673      band   88.000000   88.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
7511     song    1.440744   32.919071   Topic5  -6.9402   0.6879
6212    power    1.757062  235.211796   Topic5  -6.7417  -1.0801
7208     seri    1.896075  584.927007   Topic5  -6.6655  -1.9149
5678      one    1.724659  245.271103   Topic5  -6.7603  -1.1405
6640   releas    1.654549  243.036569   Topic5  -6.8018  -1.1729

[348 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
87        1  0.894556    adapt
87        2  0.048354    adapt
87        3  0.024177    adapt
87        4  0.024177    adapt
101       3  0.849495   admiss
...     ...       ...      ...
9105      4  0.011450  zealand
9117      1  0.058980     zone
9117      2  0.117960     zone
9117      3  0.648779     zone
9117      4  0.117960     zone

[853 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 2, 5, 1])

In [112]:
p2

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.001522  0.002909       1        1  78.198315
4     -0.000128 -0.001295       2        1   8.397453
1     -0.000845 -0.000787       3        1   6.731572
2     -0.000805 -0.000107       4        1   5.209479
3      0.000256 -0.000719       5        1   1.463181, topic_info=         Term      Freq     Total Category  logprob  loglift
7244   shadow  0.000000  0.000000  Default  30.0000  30.0000
6640   releas  0.000000  0.000000  Default  29.0000  29.0000
190     album  0.000000  0.000000  Default  28.0000  28.0000
2963     film  0.000000  0.000000  Default  27.0000  27.0000
4841    manga  0.000000  0.000000  Default  26.0000  26.0000
...       ...       ...       ...      ...      ...      ...
6640   releas  0.001124  0.182168   Topic5  -8.8973  -0.8637
340      anim  0.001074  0.103693   Topic5  -8.9425  -0.3454
5045  mendoza  0.001066  0.081666   Topic5  -8.9505  -0.1146
985      book  0.001077  0.149217   Topic5  -8.9402  -0.7071
8220  tolkien  0.001072  0.322508   Topic5  -8.9443  -1.4819

[256 rows x 6 columns], token_table=Empty DataFrame
Columns: [Topic, Freq, Term]
Index: [], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 2, 3, 4])

We observe that 3:
* a
* b
* c